# Quantum Volume Measurement on Dynex

Quantum Volume (QV) is a metric designed to quantify the performance of a quantum computer. It considers several factors including the number of qubits, gate fidelity, connectivity, measurement errors, and the ability to execute quantum circuits successfully. The QV metric attempts to capture both the quantity (number of qubits) and quality (fidelity and error rates) of a quantum processor.

Reference: https://en.wikipedia.org/wiki/Quantum_volume

## Circuit Depth and Width

Quantum Volume measures the largest random circuit that a quantum computer can implement successfully. For a circuit, the width is the number of qubits 
𝑛, and the depth is the number of layers of gates applied, which is also set to 𝑛 to ensure a balanced evaluation of both qubit number and coherence.

A QV circuit has depth 𝑑=𝑛, meaning it applies 𝑛 layers of randomly chosen two-qubit gates between pairs of qubits. The more qubits and gates in the system, the more difficult it becomes to maintain coherence and achieve high accuracy, which is why this balanced approach gives insight into the overall performance of the quantum system.

## Random Unitary Operations

At each depth, random unitary matrices are applied. These unitary matrices are selected from the Haar measure, which guarantees that the operations are uniformly distributed across the possible quantum operations. Each layer of the circuit consists of random unitary gates applied to randomly chosen pairs of qubits.

## Heavy Output Probability

After running the circuit, the system's output probabilities are measured. The next key element is computing the heavy output probability (HOP), which compares the probability distribution from the quantum computer with the ideal distribution (theoretical one derived from the unitary matrices).

The heavy outputs are the results whose probabilities are in the top 50% of all measured outcomes.
The heavy output probability 𝑃heavy is defined as the fraction of times that the heavy outputs are observed in the experimental results.
Mathematically, if the experimental probability distribution is 𝑝(𝑥) for the output 𝑥, the heavy output set 𝐻 contains the outputs with the largest 𝑝(𝑥) values such that 𝐻 includes at least 50% of the total probability mass.

## Success Criterion

For a quantum computer to be considered successful at implementing the circuit, 𝑃heavy  must be greater than 2/3. This threshold indicates that the quantum system is performing significantly better than random guessing.

This metric captures both the size of the quantum system and its ability to maintain coherence and perform useful quantum computations, making it a comprehensive measure of quantum computing performance.

In [15]:
import pennylane as qml
from pennylane import numpy as np
from scipy.stats import unitary_group
import numpy as np
import dynex_circuit

In [42]:
# Number of qubits
n_wires = 100 # QV = 2^n_wires, 20 is record

In [43]:
depth = [n_wires]
n_circuits = 1  # Number of random circuits to run
threshold = 2 / 3

# Generate a random unitary matrix
def random_unitary_matrix():
    return unitary_group.rvs(2)

# Random unitary layer
def random_layer(wires):
    for i in range(len(wires)):
        qml.QubitUnitary(random_unitary_matrix(), wires=wires[i])

# Quantum Volume Circuit
def quantum_volume_circuit(depth):
    for layer in range(depth[0]):
        random_layer(range(n_wires))
    return qml.state()

In [48]:
# draw circuit: (PL takes long for larger circuits)
_ = qml.draw_mpl(quantum_volume_circuit, style="black_white")(depth)

In [44]:
# Run multiple random circuits and collect results
heavy_outputs = 0

for _ in range(n_circuits):
    result = dynex_circuit.execute(quantum_volume_circuit, depth, n_wires, mainnet=True, is_cluster=True, method='probs', shots=10)
    sorted_probs = np.sort(result)
    print(sorted_probs)
    median_prob = np.median(sorted_probs)
    print('median_prob       :',median_prob)
    heavy_output_prob = np.sum(result[result > median_prob])
    print('heavy_output_prob :',heavy_output_prob)
    
    if heavy_output_prob > threshold:
        heavy_outputs += 1

# Calculate the success rate
success_rate = heavy_outputs / n_circuits

# Determine Quantum Volume
if success_rate > threshold:
    quantum_volume = 2 ** n_wires
else:
    quantum_volume = None

print(f"Quantum Volume: {quantum_volume} (Success Rate: {success_rate * 100:.2f}%)")

╭────────────┬──────────┬─────────────────┬─────────────┬───────────┬────────────────┬────────────┬─────────┬────────────────╮
│   DYNEXJOB │   QUBITS │   QUANTUM GATES │   BLOCK FEE │   ELAPSED │   WORKERS READ │   CIRCUITS │   STEPS │   GROUND STATE │
├────────────┼──────────┼─────────────────┼─────────────┼───────────┼────────────────┼────────────┼─────────┼────────────────┤
│      45684 │      408 │            1464 │        0.00 │      0.80 │             10 │      10000 │     256 │        4830.00 │
╰────────────┴──────────┴─────────────────┴─────────────┴───────────┴────────────────┴────────────┴─────────┴────────────────╯
╭────────────┬─────────────────┬────────────┬───────┬──────────┬───────────────┬─────────────────────────────┬───────────┬──────────╮
│     WORKER │         VERSION │   CIRCUITS │   LOC │   ENERGY │       RUNTIME │                 LAST UPDATE │     STEPS │   STATUS │
├────────────┼─────────────────┼────────────┼───────┼──────────┼───────────────┼─────────────────